# Feature Engineering

*In which we boost, combine, split, or otherwise manipulate the features of crabs.*


### Define Constants


In [7]:
DATASET_FILE = '../datasets/CrabAgePrediction.csv' # 'https://www.kaggle.com/sidhus/crab-age-prediction/download' or './data/CrabAgePrediction.csv'
CACHE_FILE = '../cache/crabs.json'
PREDICTION_TARGET = 'Age'    # 'Age' is predicted
DATASET_COLUMNS = ['Sex','Length','Diameter','Height','Weight','Shucked Weight','Viscera Weight','Shell Weight',PREDICTION_TARGET]
REQUIRED_COLUMNS = [PREDICTION_TARGET]


### Importing Libraries


In [3]:
import numpy as np
import pandas as pd

#from sklearn.svm import SVC
#from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

try:
    # for visual mode. `pip install -e .[visual]`
    import matplotlib.pyplot as plt
    import matplotlib
    %matplotlib inline
    import seaborn as sns
except ModuleNotFoundError:
    plt = None
    sns = None

pd.set_option('mode.copy_on_write', True)
pd.set_option('future.no_silent_downcasting', True)


### Feature Significance

Remove features with low variance. These features are likely to be less important for the model.


In [4]:
from sklearn.feature_selection import VarianceThreshold


### Data Augmentation

Crabs are complex creatures. Let's engineer some features to help our model find the best crabs for harvest.

We'll need to use domain knowledge to extract more features from our dataset's column.

![This kills the crab.](https://i.kym-cdn.com/photos/images/newsfeed/000/112/843/killcrab.jpg)

For example, we can find the edible weight of the crab by subtracting the viscera weight from the shucked weight.


In [5]:
def data_augmentation(df:pd.DataFrame) -> pd.DataFrame:
    """Add new features to the DataFrame.

    Driven by domain knowledge.

    :param df: The data.
    :return: The data with new features.
    """
    # add new features by combining existing features
    df['Edible Weight'] = df['Shucked Weight'] - df['Viscera Weight']
    return df


### Memory Reduction

Crabs were never known for their memory. Let's minimize the memory of our DataFrame using the smallest data types to fit the data.

The reason for this is to save computational resources and time. The smaller the data, the faster the processing.


In [6]:
def data_downcasting(df:pd.DataFrame) -> pd.DataFrame:
    """Reduce the memory usage of the DataFrame by downcasting numeric types.
    
    :param df: DataFrame to be reduced.
    :return: DataFrame with reduced memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.4f} MB (before)')

    # Define mapping of types to numpy types
    type_ranges = {
        'int': [(np.iinfo(np.int8).min, np.iinfo(np.int8).max, np.int8),
                (np.iinfo(np.int16).min, np.iinfo(np.int16).max, np.int16),
                (np.iinfo(np.int32).min, np.iinfo(np.int32).max, np.int32),
                (np.iinfo(np.int64).min, np.iinfo(np.int64).max, np.int64)],
        'float': [(np.finfo(np.float16).min, np.finfo(np.float16).max, np.float16),
                  (np.finfo(np.float32).min, np.finfo(np.float32).max, np.float32),
                  (np.finfo(np.float64).min, np.finfo(np.float64).max, np.float64)]
    }

    for col in df.columns:
        col_type = df[col].dtype
        c_min, c_max = df[col].min(), df[col].max()

        if col_type in ['int16', 'int32', 'int64']:
            for min_val, max_val, new_type in type_ranges['int']:
                if c_min > min_val and c_max < max_val:
                    df[col] = df[col].astype(new_type)
                    break

        elif col_type in ['float16', 'float32', 'float64']:
            for min_val, max_val, new_type in type_ranges['float']:
                if c_min > min_val and c_max < max_val:
                    df[col] = df[col].astype(new_type)
                    break

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {end_mem:.4f} MB (after)')
    print(f'Reduced {100 * (start_mem - end_mem) / start_mem:.1f}%')

    return df


### Onwards to Final Evaluation

See the [next section](../3-evaluation/evaluation.ipynb) for the final evaluation.
